### misit 과학기술 정보 통신부
-http://msit.go.kr
- 보도자료를 크롤링
- json으로 데이터 수집
- json 데이터 안에 gtml 코드가 들어있습니다

### 이터레이터, 재너레이터
- iterable : 순서가 있는 데이터의 집합
- iterator : next 함수를 이용하여 값을 생성하는 값 생성기
- geverator : iterator를 간결하게 만든 문법 

# 1. 간단한 이터레이터 만들기
data=[1,2,3]
iterator = iter(data)
type(iterator), iterator


In [ ]:
next(iterator)

In [ ]:
#2. 피보나치수열 itorator 로 구현
class Fib:
    def __init__(self):
        self.prev = 0
        self.curr = 1
    def __iter__(self):
        return self
    def __next__(self):
        value = self.curr
        self.curr += self.prev
        self.prev = value
        return value

In [ ]:
fib = Fib()

In [2]:
next(fib)

NameError: name 'fib' is not defined

In [ ]:
### 3. 제너레이터 
#이터레이터를 간결하게 구현할 수 있는 문법
#클래스로 만드는 대신 함수로 생성할 수 있다
#yifeld 예약어 사용

In [ ]:
def fib():
    prev, curr = 0,1
    while True:
        yield curr#값은 리턴하고 코드실행이 중단
        prev, curr = curr, prev + curr
            


In [ ]:
f=fib()

In [ ]:
next(f)

In [ ]:
def test():
    yield 1
    yield 2
    yield 3

In [ ]:
t = test()
type(t)

In [ ]:
next(t)

In [ ]:
#리스트 컴프리헨션 : 제너레이터의 한 종류

In [ ]:
data = [1,2,3]
[data*data for data in data]


In [ ]:
result= {data:data**2 for data in data}
type(result), result

In [ ]:
#tuple 컴프리헨션으로 제너레이터 만들기

In [ ]:
#튜플로 하면 제너레이터가된다
gen = (data**2 for data in data)
type(gen), gen

In [ ]:
next(gen)

In [ ]:
# ### scrapy
# -scraoy 프로젝트 만들기
# - scrapy 기본구조
# - scrapy 코드작성
# - scrapy 실행
!pip install scrapy

In [ ]:
# 1. 스크레피 프로젝트 생성
!scrapy startproject crawler

In [ ]:
!ls

In [ ]:
!tree crawler

In [ ]:
#2. scrapy 의 기본구조 

In [ ]:
- spiders 디렉토리
    -어떤 사이트를 어떤 절차로 크롤링할 것인지를 코드로 명시하는 디렉토리
    -여러개의 spider를 모듈로 생성할 수 있다
-items.py
    -웹페이지에서 원하는 데이터를 저장할 때 사용되는 자료구조 클래스
-pipelines.py
    - 크롤링한 데이터를 처리하는 코드를 작성 
-settings.py
    - 스크래피 프로젝트에 대한 설정값 : rebots.txt를 따를지 안따를지 설정 ... 

In [27]:
###3. scrapy 코드 작성
# 상품 링크 크롤링 > 상세페이지에서 제목,판매가,원가 크롤링
from scrapy.http import TextResponse
req = requests.get("http://corners.gmarket.co.kr/Bestsellers")
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 http://corners.gmarket.co.kr/Bestsellers>

In [28]:
links = response.xpath(
    '//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li/div[1]/a/@href').extract()
links[-3:]

['http://item.gmarket.co.kr/Item?goodscode=1825372757&ver=637293895192749189',
 'http://item.gmarket.co.kr/Item?goodscode=373058782&ver=637293895192749189',
 'http://item.gmarket.co.kr/Item?goodscode=1114973997&ver=637293895192749189']

In [29]:
# 절차
#1. 웹페이지 분석 및 selector(xpath) 찾기 
#2. scrapy 프로젝트 생성 및 코드 작성(items> spider > pipeline)
#3. scrapy 실행(크롤링)
link=links[2]


In [30]:
# 베스트 셀러 상품 200개 링크 selector 
req=requests.get(link)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 http://item.gmarket.co.kr/Item?goodscode=1835382457&ver=637293895192593185>

In [31]:
# 상세페이지에서 제목, 판매가, 원가 수집
title = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract().strip()
s_price = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract().strip()

try: 
    o_price = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract().strip()
except:
    o_price = s_price
#o_price
#s_price
title, s_price, o_price

('블랑슈 센시티브 아기물티슈 캡형 100매x10팩', '9,900', '27,500')

In [32]:
#프로젝트 만들기
! scrapy startproject gmarket 


Error: scrapy.cfg already exists in /Users/yeonghwanchoi/Documents/dev/TIL/linear_regression_project/gmarket


In [33]:
#1.item.py 코드작성: 제목 상품링크,원가 판매가


In [54]:
# %load gmarket/gmarket/items.py
# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class GmarketItem(scrapy.Item):
    
    title = scrapy.Field()
    link = scrapy.Field()
    o_price = scrapy.Field()
    s_price = scrapy.Field()
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


In [55]:
#2. spider.py 코드작성

In [64]:
%%writefile gmarket/gmarket/spiders/spiders.py
import scrapy
from gmarket.items import GmarketItem

class Spider(scrapy.Spider):#여러게 크롤링하고 싶다면 스파이더 추가
    
    name = "GmarketBestSellers"
    allow_domain=["gmarket.co.kr"]
    start_urls= ["http://corners.gmarket.co.kr/Bestsellers"] #처음 시작하는 url
    
    def parse(self, response):
        links = response.xpath(
            "//*[@id='gBestWrap']/div/div[3]/div[2]/ul/li[1]/div[1]/a/@href").extract()
        for link in links: 
            yield scrapy.Request(link, callback=self.parse_content)#yield 
    def parse_content(self, response):
        item = GmarketItem()
        item["title"] = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract().strip()
        item["s_price"]=response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract().strip()
        
        try:
            item["o_price"]=response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract().strip()
        except:
            item["o_price"]=item["s_price"]
        item["link"]= response.url
        yield item

Overwriting gmarket/gmarket/spiders/spiders.py


In [65]:
# title = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract().strip()
# s_price = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract().strip()

# try: 
#     o_price = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract().strip()
# except:
#     o_price = s_price
# #o_price
# #s_price
# title, s_price, o_price

In [66]:
#3. scrapy 실행
# scrapy.cfg 파일이 있는 디렉토리에서 $scrapy crawl <project name><spider name> 으로 실행
#$cd gmarket
#$scrapy gmarket GmarketBestsellers


In [67]:
%%writefile run.sh
cd gmarket
scrapy crawl GmarketBestSellers

Overwriting run.sh


In [68]:
#windows : git bash에서 직접실행
#mac :!source run.sh
#ubuntu: ./run.sh

In [69]:
!chmod 777 run.sh

In [70]:
!source run.sh

2020-07-03 16:20:15 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: gmarket)
2020-07-03 16:20:15 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Darwin-19.5.0-x86_64-i386-64bit
2020-07-03 16:20:15 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-07-03 16:20:15 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'gmarket',
 'NEWSPIDER_MODULE': 'gmarket.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['gmarket.spiders']}
2020-07-03 16:20:15 [scrapy.extensions.telnet] INFO: Telnet Password: abf9abfc3ab3b9a5
2020-07-03 16:20:15 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',

In [19]:
#결과 csv로 저장
    

In [20]:
%%writefile run.sh
cd gmarket
scrapy crawl GmarketBestsellers -o gmarket200.csv

Overwriting run.sh


In [21]:
!source run.sh

Traceback (most recent call last):
  File "/Users/yeonghwanchoi/opt/anaconda3/bin/scrapy", line 8, in <module>
    sys.exit(execute())
  File "/Users/yeonghwanchoi/opt/anaconda3/lib/python3.7/site-packages/scrapy/cmdline.py", line 142, in execute
    cmd.crawler_process = CrawlerProcess(settings)
  File "/Users/yeonghwanchoi/opt/anaconda3/lib/python3.7/site-packages/scrapy/crawler.py", line 280, in __init__
    super(CrawlerProcess, self).__init__(settings)
  File "/Users/yeonghwanchoi/opt/anaconda3/lib/python3.7/site-packages/scrapy/crawler.py", line 152, in __init__
    self.spider_loader = self._get_spider_loader(settings)
  File "/Users/yeonghwanchoi/opt/anaconda3/lib/python3.7/site-packages/scrapy/crawler.py", line 146, in _get_spider_loader
    return loader_cls.from_settings(settings.frozencopy())
  File "/Users/yeonghwanchoi/opt/anaconda3/lib/python3.7/site-packages/scrapy/spiderloader.py", line 68, in from_settings
    return cls(settings)
  File "/Users/yeonghwanchoi/opt/anac

In [22]:
!ls gmarket/

gmarket    scrapy.cfg


In [24]:
pd.read_csv("gmarket/gmarket200.csv")

FileNotFoundError: [Errno 2] File gmarket/gmarket200.csv does not exist: 'gmarket/gmarket200.csv'